In [1]:
import pandas as pd
import urllib.request

changes made 5/30: (feel free to look at github history if you want to revert some stuff)
- loaded in court_list and processed judges outside of function
- loaded in detail_codes for eventual mapping 
- used map instead of merge to replace court id with court name outside of function - looks cleaner
- took out all the stuff with the judges dataframe since it's already been moved over to the script
- creating a function for mapping codes with actual meanings
    - started the probation type column b/c "No Probation" and "NP" was my mistake when scraping

In [2]:
#read in courts and judges
court_list = pd.read_csv("https://raw.githubusercontent.com/christinegu27/State-Sentencing-Project/main/CSV%20Processing/courts.csv")
judges = pd.read_csv("https://raw.githubusercontent.com/christinegu27/State-Sentencing-Project/main/CSV%20Processing/judges.csv")
codes = pd.read_csv("https://raw.githubusercontent.com/christinegu27/State-Sentencing-Project/main/CSV%20Processing/detail_codes.csv")
#dictionary for replacing the court id with the court name
court_code = dict(court_list[["Court ID", "Court Name"]].values)


In [3]:
#Replace codes in the dataframe with their actual meaning
#Ex. replace "W" with "White" for defendant race
def map_values(df):
    d = dict()
    f = open("/Users/hinaljajal/Downloads/State-Sentencing-Project/CSV Processing/detail_codes.csv")

    for line in f:
        line = line.strip('\n')
        line = line.split(",")
        key,val=line[0],line[1]
        d[key] = val
    df = df.replace(d)
    return df

I'm thinking it might be better to load in the stuff outside the function when generalizing if we're eventually going to concatenate them all together

In [12]:
def case_matcher(cases):
    """
    cases: a dataframe of cases for 1 specifc court
    """
#     case_url="/Users/hinaljajal/Downloads/{}.csv".format(court+"_cases")
#     cases=pd.read_csv(case_url)
    
    cases["Year"]=cases["Last Hearing Date"].str[-4:]

    #checks if there are are different judges with the same initials
    judges1 = judges.drop_duplicates(subset=['Judge Full Name'])
    boolean = judges1.duplicated(subset=['Judge']).any()
    
    #Merges the columns by the judge and the hearing year
    cases['Year']=cases['Year'].astype(int)
    cases['Judge']=cases['Judge'].astype(str)
    judges['Year']=judges['Year'].astype(int)
    judges['Judge']=judges['Judge'].astype(str)


    cases = cases.merge(judges, how="left",on = ["Judge", "Year"])
        
    #Data cleaning
    cases['Race'] = cases['Race'].fillna("U") #U is the code for unknowns 
    cases['Sentence Y'] = cases['Sentence Y'].fillna(0)
    cases['Sentence M'] = cases['Sentence M'].fillna(0)
    cases['Sentence D'] = cases['Sentence D'].fillna(0)
    cases['Probation D'] = cases['Probation D'].fillna(0)
    cases['Probation Y'] = cases['Probation Y'].fillna(0)
    cases['Probation M'] = cases['Probation M'].fillna(0)
    cases['Judge Full Name'] = cases['Judge Full Name'].fillna("Unknown") #Nans are filled with unknowns 


    #Saves the dataframe to a csv
    file_name = '/Users/hinaljajal/Downloads/{}.csv'.format(court)
    cases.to_csv(file_name, index = False)
    
    return cases

In [13]:
separate_court_data = []
for court in court_list["Court ID"]:
    try:
        data = pd.read_csv("/Users/hinaljajal/Downloads/finished_courts/{}.csv".format(court))
    except FileNotFoundError:
        continue
    data = map_values(data)
    #Slices the judges for the particular court 
    judges[judges["Court Name"]==court_code[court]]
    #replace court code with name, ie "001C" with "Accomack"
    data["Court"] = court_code[court]
    data = case_matcher(data)
    separate_court_data.append(data) #add to list for concatenation
    
final_cases = pd.concat(separate_court_data, ignore_index = True)

/Users/hinaljajal/Downloads/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [21]:
#put in database?
final_cases["Case Number"]=final_cases["Case Number"].astype(str)

In [22]:
final_cases.head()

,Case Number,Name,Court,Last Hearing Date,Charge,Charge Code,Charge Class,Charge Code Section,Concluded By,Sentence Y,...,Probation Type,Probation Y,Probation M,Probation D,Race,Gender,Judge,Year,Judge Full Name,Court Name
0,1700020702,"CORBIN, ELISHA AZEL",Accomack,05/21/2020,VIOL PROBATION OF MISD OFF,TRAVEL TO AND FROM JAIL WHEN DEFENDANT IS ON W...,UNSUPERVISED,19.2-306,TRIAL - JUDGE WITH WITNESS,0.0,...,SUPERVISED,2.0,0.0,0.0,BAIL,TRAVEL TO AND FROM JAIL WHEN DEFENDANT IS ON W...,WRL,2020,William Revell Lewis,Northampton
1,1700020702,"CORBIN, ELISHA AZEL",Accomack,05/21/2020,VIOL PROBATION OF MISD OFF,TRAVEL TO AND FROM JAIL WHEN DEFENDANT IS ON W...,UNSUPERVISED,19.2-306,TRIAL - JUDGE WITH WITNESS,0.0,...,SUPERVISED,2.0,0.0,0.0,BAIL,TRAVEL TO AND FROM JAIL WHEN DEFENDANT IS ON W...,WRL,2020,William Revell Lewis,Accomack
2,1700020702,"CORBIN, ELISHA AZEL",Accomack,05/21/2020,VIOL PROBATION OF MISD OFF,TRAVEL TO AND FROM JAIL WHEN DEFENDANT IS ON W...,UNSUPERVISED,19.2-306,TRIAL - JUDGE WITH WITNESS,0.0,...,SUPERVISED,2.0,0.0,0.0,BAIL,TRAVEL TO AND FROM JAIL WHEN DEFENDANT IS ON W...,WRL,2020,William Revell Lewis,Virginia Beach
3,0900010001,"AYRES, UPSHUR",Accomack,02/25/2010,DISTRIBUTION OF COCAINE,PROTECTIVE ORDER - FINAL,NaN,18.2-248,TRIAL - JUDGE WITH WITNESS,5.0,...,INDEFINITE SUPERVISION,2.0,0.0,0.0,BAIL,TRAVEL TO AND FROM JAIL WHEN DEFENDANT IS ON W...,GAT,2010,Glen A Tyler,Northampton
4,0900010001,"AYRES, UPSHUR",Accomack,02/25/2010,DISTRIBUTION OF COCAINE,PROTECTIVE ORDER - FINAL,NaN,18.2-248,TRIAL - JUDGE WITH WITNESS,5.0,...,INDEFINITE SUPERVISION,2.0,0.0,0.0,BAIL,TRAVEL TO AND FROM JAIL WHEN DEFENDANT IS ON W...,GAT,2010,Glen A Tyler,Accomack
